<a href="https://colab.research.google.com/github/mikexcohen/Calculus_book/blob/main/ch15_improper_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Calculus unraveled: Intuition, Proofs, and Python**
### Mike X Cohen (sincxpress.com)
#### https://github.com/mikexcohen/calculus_book
#### Code for Chapter 15 (Improper integrals)

---

# About this code file:

### This notebook contains full code solutions to the exercises in this book chapter. There are many correct ways to solve the exercises; this notebook provides *a* solution, not *THE* solution. Please use this code as a starting point to continue exploring and experimenting with calculus concepts and visualizations.

## **Using the code without the book may lead to confusion or errors.**

#### This code was written in google-colab. The notebook may require some modifications if you use a different IDE.

In [ ]:
# import libraries and define global settings
import numpy as np
import sympy as sym
import math
import scipy.integrate as spi
import matplotlib.pyplot as plt

from IPython.display import Math

# define global figure properties used for publication
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg') # display figures in vector format
plt.rcParams.update({'font.size':14,             # font size
                     'savefig.dpi':300,          # output resolution
                     'axes.titlelocation':'left',# title location
                     'axes.spines.right':False,  # remove axis bounding box
                     'axes.spines.top':False,    # remove axis bounding box
                     'lines.linewidth':2         # increase default line thickness
                     })

# Exercise 1: Sympy for improper integrals

In [ ]:
x = sym.symbols('x')

#          function     lower limit    upper limit
funs = [ [ 1/x**2      ,    1       ,    sym.oo  ],
         [ sym.pi/x**3 ,    1       ,    sym.oo  ],
         [ 1/sym.sqrt(x),   1       ,    sym.oo  ],
         [ sym.Abs(sym.cos(x)),0    ,    sym.oo  ],
         [ sym.cos(x)  ,    0       ,    sym.oo  ],
         [ sym.cos(x**2),   0       ,    sym.oo  ]
]

# calculate and print their integral
for fi in range(len(funs)):

  # extract the information
  f = funs[fi][0]
  limL = funs[fi][1]
  limU = funs[fi][2]

  # compute the integral
  int2display = sym.Integral(f,(x,limL,limU)) # for visualization
  defint = sym.integrate(f,(x,limL,limU))

  # and display
  display(Math('%s = %s' % (sym.latex(int2display),sym.latex(defint))))
  print('')

# Exercise 15.2: A range of p's

In [ ]:
# the range of p-values
pvalues = np.linspace(-3,1,21)

# range of x-axis grid locations for plotting
xx = np.linspace(.01,10,301)


# loop over exponents
_,axs = plt.subplots(1,2,figsize=(12,4))
for p in pvalues:

  # get the indefinite integral of the function
  F = sym.integrate(x**p) # symbolic 'x' created in ex.1

  # use FTC-2 to determine whether it converges or diverges
  defint = F.subs(x,sym.oo) - F.subs(x,1)

  # pick color based on integral
  c='k' if defint.is_finite else [.6,.6,.6]

  # lambdify and visualize
  F_l = sym.lambdify(x,F)
  axs[0].plot(xx,F_l(xx),color=c)

  # plot the convergence
  axs[1].plot(p,1-defint.is_finite,'s',color=c,markersize=12)


axs[0].set(xlabel='$x$',xlim=xx[[0,-1]],ylabel='$Y = F(x)$',ylim=[-15,30],title=r'$\bf{A}$)  Indefinite integral functions')
axs[1].set(yticks=[0,1],yticklabels=['Converges','Diverges'],xlabel=r'$p$ in $x^{p}$',ylim=[-.5,1.5],title=r'$\bf{B}$)  Definite integral convergence')

plt.tight_layout()
plt.savefig('improper_ex2.png')
plt.show()

# Exercise 15.3: Examining convergence

In [ ]:
from sympy.abc import x

# range of exponent values
pExponents = np.linspace(-3,-1.5,21)
colors = np.linspace(0,.9,len(pExponents))

# x-axis grid values
xx = np.linspace(1,10,301)


# define the upper limit of integration (lower bound is hard-coded to 1)
upperLims = np.logspace(np.log10(2),np.log10(1e5),43)
integrals = np.zeros(len(upperLims))

# define a colormap for the lines
fig,axs = plt.subplots(1,2,figsize=(12,4))

# loop over exponents, calculate, plot
for p,c in zip(pExponents,colors):

  # create lambda function
  fx_l = sym.lambdify(x,x**p)

  # plot the function
  axs[0].plot(xx,fx_l(xx),color=np.full(3,c))

  # compute empirical integrals
  for ui in range(len(upperLims)):
    integrals[ui] = spi.quad(fx_l,1,upperLims[ui])[0]

  # plot the the definite integral
  axs[1].plot(upperLims,integrals,color=np.full(3,c))


# add the colorbar
import matplotlib as mpl
norm = mpl.colors.Normalize(vmin=pExponents[0],vmax=pExponents[-1])
cmap = mpl.cm.ScalarMappable(norm=norm,cmap=mpl.cm.gray)
cmap.set_array([])

cbar = plt.colorbar(cmap,ax=axs[1],ticks=pExponents[::3])
cbar.set_label(r'Exponent (p in $x^p$)')

# make the plots look nicer
axs[0].set(xlim=xx[[0,-1]],xlabel='$x$',ylabel='$y = f(x)$',title=r'$\bf{A}$)  Function curves')
axs[1].set(xlim=upperLims[[0,-1]],xscale='log',xlabel='Upper bound',
           ylabel='Definite integral (A)',title=r'$\bf{B}$)  Area under $f(x)=x^p$')

plt.tight_layout()
plt.savefig('improper_ex3.png')
plt.show()

# Exercise 15.4: A sined Morlet

In [ ]:
# Let's try:
theta = sym.symbols('theta')

f = sym.sin(theta) * sym.exp(-theta**2)
sym.integrate(f,(theta,-sym.oo,sym.oo))

In [ ]:
# function
th = np.linspace(-np.pi,np.pi,5001)
f = np.sin(2*np.pi*th) * np.exp(-th**2)

# definite integral
defint = spi.trapezoid(f,dx=th[1]-th[0])

In [ ]:
# and plot
_,ax = plt.subplots(1,figsize=(6,4))

# plot one wavelet
ax.plot(th,f,'k',linewidth=1)
ax.fill_between(th,f,color='k',alpha=.2)
ax.set(xlim=th[[0,-1]],xlabel=r'$\theta$',ylabel=r'$y=f(\theta)$',
           xticks=np.arange(-np.pi,np.pi+.1,step=np.pi/2),xticklabels=['$-\pi$','$-\pi/2$',0,'$\pi/2$','$\pi$'])
ax.set_title(fr'A $\approx$ {defint:.25f}')

# finalize
plt.tight_layout()
plt.savefig('improper_ex4.png')
plt.show()

# Exercise 15.5

In [ ]:
# initialize
frex = np.linspace(.5,4*np.pi,30)
defAreas = np.zeros((len(frex),2))

for fi in range(len(frex)):

  # create the wavelets
  f_c = np.cos(frex[fi]*th) * np.exp(-th**2)
  f_s = np.sin(frex[fi]*th) * np.exp(-th**2)

  # approximate areas
  defAreas[fi,0] = spi.trapezoid(f_s,dx=th[1]-th[0])
  defAreas[fi,1] = spi.trapezoid(f_c,dx=th[1]-th[0])


# and plot
_,ax = plt.subplots(1,figsize=(7,4))

# plot the approximated area by sine frequency
ax.plot(frex,defAreas[:,0],'ks-',markersize=8,label='Area of sine wavelets')
ax.plot(frex,defAreas[:,1],'o-',color=[.7,.7,.7],markersize=8,label='Area of cosine wavelets')
ax.legend()
ax.set(xlabel='Frequency (rad.)',ylabel='Approximate area',xlim=[frex[0]-.2,frex[-1]+.2])

# finalize
plt.tight_layout()
plt.savefig('improper_ex5.png')
plt.show()

# Exercise 15.6: More wavelet explorations

In [ ]:
frex = np.linspace(np.pi/10,6*np.pi,30)
defint = np.zeros((len(frex),2))

xx = np.linspace(-20*np.pi,20*np.pi,5001)

_,axs = plt.subplots(1,2,figsize=(12,4))

for fr in range(len(frex)):

  # create function
  f = np.cos(frex[fr]*xx) * np.exp(-xx**2)

  # get net and total areas
  defint[fr,0] = spi.trapezoid(f,dx=xx[1]-xx[0])
  defint[fr,1] = spi.trapezoid(abs(f),dx=xx[1]-xx[0])

  # plot selected functions
  if fr%9==0:
    axs[0].plot(xx,f,color=np.full(3,fr*.8/len(frex)))


# plot
axs[1].plot(frex,defint[:,1],'ko-',markerfacecolor=[.8,.8,.8],markersize=7,label='Total area')
axs[1].plot(frex,defint[:,0],'ks-',markerfacecolor='w',markersize=7,label='Net area')
axs[1].legend()

axs[0].set(xlim=[-2,2],xlabel=r'$\theta$',ylabel=r'$y=f(\theta)$',title=r'$\bf{A}$)  Some Morlet wavelets')
axs[1].set(xlabel='Cosine frequency (rad.)',ylabel='Area',title=r'$\bf{B}$)  Areas by frequency')

plt.tight_layout()
plt.savefig('improper_ex6.png')
plt.show()

# Exercise 15.7:

In [ ]:
x = sym.symbols('x')
f1 = 1 / (sym.sqrt(x) * (2+sym.sin(x)) )
f2 = 1 / (x**2 * (2+sym.sin(x)) )

display(Math('f_1(x) = %s, \quad F_1(x) = %s' %(sym.latex(f1),sym.latex(sym.integrate(f1)))))
print('')
display(Math('f_2(x) = %s, \quad\; F_2(x) = %s' %(sym.latex(f2),sym.latex(sym.integrate(f2)))))

In [ ]:
# numerically evaluate the functions and their integrals
xx = np.linspace(np.pi,10*np.pi,500)

f1_y = np.array( [f1.subs(x,xi) for xi in xx] )
f2_y = np.array( [f2.subs(x,xi) for xi in xx] )

# for the final part of the exercise:
#f1_y /= np.max(f1_y)
#f2_y /= np.max(f2_y)

f1i_y = spi.cumulative_trapezoid(f1_y,dx=xx[1]-xx[0],initial=0) # "initial" includes an additional value at the start
f2i_y = spi.cumulative_trapezoid(f2_y,dx=xx[1]-xx[0],initial=0)


# draw the functions
_,axs = plt.subplots(1,2,figsize=(12,4))
axs[0].plot(xx,f1_y,'--',color=[.7,.7,.7],label=r'$f_1(x)$')
axs[0].plot(xx,f2_y,'k',label=r'$f_2(x)$')
axs[0].set(xlim=xx[[0,-1]],xlabel='$x$',ylabel=r'$y = f(x)$',title=r'$\bf{A}$)  The functions')
axs[0].legend()

# the integrals
axs[1].plot(xx,f1i_y,'--',color=[.7,.7,.7],label=r'$F_1(x)$')
axs[1].plot(xx,f2i_y,'k',label=r'$F_2(x)$')
axs[1].set(xlim=xx[[0,-1]],xlabel='$x$',ylabel=r'$Y = F(x)$',title=r'$\bf{B}$)  Their numerical antiderivatives')
axs[1].legend()

plt.tight_layout()
plt.savefig('improper_ex7b.png')
plt.show()

In [ ]:
# their integrals
display(Math('%s \;=\; %s \;\\approx\; %s'
             %(sym.latex(sym.Integral(f1,(x,sym.pi,sym.oo))),sym.latex(sym.integrate(f1,(x,sym.pi,sym.oo))),sym.latex(sym.integrate(f1,(x,sym.pi,sym.oo)).evalf()))))
print('')
display(Math('%s \;=\; %s \;\\approx\; %s'
             %(sym.latex(sym.Integral(f2,(x,sym.pi,sym.oo))),sym.latex(sym.integrate(f2,(x,sym.pi,sym.oo))),sym.latex(sym.integrate(f2,(x,sym.pi,sym.oo)).evalf()))))


# Exercise 15.8

In [ ]:
x = sym.symbols('x')
f = sym.sin(x) / x

display(Math('f(x) = %s' %sym.latex(f)))
print('')
display(Math('F(x) = %s' %sym.latex(sym.integrate(f))))

In [ ]:
dx = .01
xx = np.arange(-14*np.pi,14*np.pi+dx,step=dx)
fx = np.sin(xx) / xx

fx_i = spi.cumulative_trapezoid(fx,dx=dx,initial=0)

_,axs = plt.subplots(1,2,figsize=(10,4))
axs[0].plot(xx,fx,'k')
axs[0].plot(0,1,'ko',markerfacecolor='w')
axs[0].set(xlim=xx[[0,-1]],title=r'$\bf{A}$)  The sinc function')

axs[1].plot(xx,fx_i,'k')
axs[1].set(xlim=xx[[0,-1]],title=r'$\bf{B}$)  Its numerical antiderivative')

plt.tight_layout()
plt.savefig('improper_ex8b.png')
plt.show()

In [ ]:
# recreate the sinc function with higher resolution for the narrow domain
dx = .001
xx = np.arange(-.1,1.1,step=dx)
fx = np.sin(xx) / xx

fx_i = spi.cumulative_trapezoid(fx,dx=dx,initial=0)

_,axs = plt.subplots(1,2,figsize=(10,3))
axs[0].plot(xx,fx,'k')
axs[0].plot(0,1,'ko',markerfacecolor='w')
axs[0].fill_between(xx[(xx>=0) & (xx<=1)],fx[(xx>=0) & (xx<=1)],color='k',alpha=.2)
axs[0].set(xlim=xx[[0,-1]],title=r'$\bf{A}$)  The sinc function')

axs[1].plot(xx,fx_i,'k')
axs[1].set(xlim=xx[[0,-1]],title=r'$\bf{B}$)  Its numerical antiderivative')

plt.tight_layout()
plt.savefig('improper_ex8c.png')
plt.show()

In [ ]:
# final answer
x0 = np.argmin(abs(xx))
x1 = np.argmin(abs(xx-1))

print(f'Sympy result: {sym.integrate(f,(x,0,1)).evalf()}')
print(f'Scipy result: {fx_i[x1] - fx_i[x0]}')